In [1]:
import requests

import pandas as pd

from bs4 import BeautifulSoup

from datetime import datetime

In [2]:
news_600_author = pd.read_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/news_author/cleaned_authors_updated/news_600_author_cl.csv')


In [3]:
news_600_author_1 = news_600_author.copy()


In [4]:
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup

def add_credit(dataframe_author):
    author_list = list(dataframe_author["author"])
    credit_article = ["None"] * len(author_list)
    credit_followers = ["None"] * len(author_list)
    credit_tweets = ["None"] * len(author_list)
    for i, names in enumerate(tqdm(author_list)):
        if names == "NaN":
            continue
        elif "NBA.com" in str(names) or "NBA News" in str(names) or "Press" in str(names) or "Official Release" in str(names) or "NBA Twitter" in str(names):
            credit_article[i] = "Default"
            credit_followers[i] = "Default"
            credit_tweets[i] = "Default"
        else:
            try:
                player_search = str(names).replace(" ", "-")
                url = "https://muckrack.com/" + player_search
                response = requests.get(url)
                soup = BeautifulSoup(response.text, 'html.parser')
                html = soup.find_all('a', attrs={'class':"profile-edit-btn"})
                if len(html) == 0:
                    num_articles = "None"
                    credit_article[i] = num_articles
                else:
                    for line in html:
                        if "See all" in line.text:
                            span_elem = line.find('span')
                            text_content = span_elem.text.strip()
                            num_articles = text_content.split()[0].replace(',', '')
                            credit_article[i] = num_articles
                            break
                    if credit_followers[i] == "None":
                        credit_followers[i] = "None"
                        
                    tweet = soup.find('p', attrs={'class':"fs-6 mt-0 mb-6"})
                    followers = tweet.find('strong').text.replace(',', '')
                    tweets = tweet.find_all('strong')[1].text.replace(',', '')
                    if len(followers) != 0:
                        credit_followers[i] = followers
                    if len(followers) == 0:
                        followers = "0" 
                        credit_followers[i] = followers
                    if len(tweets) != 0:
                        credit_tweets[i] = tweets
                    if len(tweets) == 0:
                        tweets = "0"
                        credit_tweets[i] = tweets
            except Exception as e:
                if credit_followers[i] == "None":
                    credit_followers[i] = "None"
                if credit_tweets[i] == "None":
                    credit_tweets[i] = "None"
    dataframe_author["credit_article"] = credit_article
    dataframe_author["credit_followers"] = credit_followers
    dataframe_author["credit_tweets"] = credit_tweets
    return dataframe_author


In [5]:
news_600_author_cedit_1 = add_credit(news_600_author_1)

100%|██████████| 6000/6000 [05:54<00:00, 16.94it/s]  


In [6]:
news_600_author_cedit_1.to_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/author_credit_nba/news_600_author_credit_1.csv', index=False)
